# Just for Alex

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

In [49]:
app_events = pd.read_csv('data/app_events.csv', nrows = 10000)
brand_df = pd.read_csv('data/phone_brand_device_model.csv')
label_categories = pd.read_csv('data/label_categories.csv')
app_labels = pd.read_csv('data/app_labels.csv')
events = pd.read_csv('data/events.csv', nrows = 10000)
gender_age_train = pd.read_csv('data/gender_age_train.csv')
gender_age_test = pd.read_csv('data/gender_age_test.csv')

In [50]:
# have a peek at these dataframes
display("app_events")
display(app_events.head())
display("brand_df")
display(brand_df.head())
display("label_categories")
display(label_categories.head())
display("app_labels")
display(app_labels.head())
display("events")
display(events.head())
display("gender_age_train")
display(gender_age_train.head())
display("gender_age_test")
display(gender_age_test.head())

'app_events'

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


'brand_df'

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


'label_categories'

...

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


'gender_age_train'

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


'gender_age_test'

,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277
3,-6220210354783429585
4,-5893464122623104785


In [51]:
# Making sense of the data: merging data frame
complete_ref_table = app_events
complete_ref_table = complete_ref_table.merge(app_labels, on = "app_id")
complete_ref_table = complete_ref_table.merge(label_categories, on = "label_id")
del complete_ref_table['is_installed']
complete_ref_table = complete_ref_table.merge(events, on="event_id")
complete_ref_table = complete_ref_table.merge(brand_df, on="device_id")
complete_ref_table.device_model = pd.Categorical(complete_ref_table.device_model)
complete_ref_table["device_model_code"] = complete_ref_table.device_model.cat.codes
complete_ref_table = complete_ref_table[["event_id", "app_id", "label_id", "category", "device_id", "device_model_code", "phone_brand", "device_model", "timestamp", "longitude", "latitude", "is_active"]]

display(complete_ref_table.head())


,event_id,app_id,label_id,category,device_id,device_model_code,phone_brand,device_model,timestamp,longitude,latitude,is_active
0,2,5927333115845830913,549,Property Industry 1.0,-6401643145415154744,118,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
1,2,8693964245073640147,549,Property Industry 1.0,-6401643145415154744,118,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
2,2,4775896950989639373,549,Property Industry 1.0,-6401643145415154744,118,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
3,2,-8022267440849930066,549,Property Industry 1.0,-6401643145415154744,118,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,0
4,2,9112463267739110219,549,Property Industry 1.0,-6401643145415154744,118,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,0


In [52]:
complete_ref_table.merge(gender_age_train, on="device_id").size

3729885

In [53]:
# Build training set
train_set = (complete_ref_table.merge(gender_age_train, on="device_id")).head(3500000)
test_set = complete_ref_table.merge(gender_age_train, on="device_id").tail(200000)
test_set.head()

,event_id,app_id,label_id,category,device_id,device_model_code,phone_brand,device_model,timestamp,longitude,latitude,is_active,gender,age,group
48659,1272,-6689096779184407258,548,Industry tag,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48660,1272,2577600876485430729,548,Industry tag,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48661,1272,-4533168039707605782,548,Industry tag,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48662,1272,-7904801204405952327,548,Industry tag,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48663,1272,2353469805335273836,548,Industry tag,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48754,1272,-1074044343920923741,209,pictures photography,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48755,1272,-1442117570005520713,209,pictures photography,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48756,1272,-653184325026622087,562,Science and Technology,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38
48757,1272,-653184325026622087,564,pursue,5022715212150420319,14,OPPO,6607,2016-05-03 01:41:30,120.44,36.17,0,M,35,M32-38


In [54]:
X_train = train_set[["label_id", "device_model_code"]].values
Y_train = train_set["gender"].values

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state = 1)
classifier.fit(X_train,Y_train)

Y_pred = classifier.predict(test_set[["label_id", "device_model_code"]].values)
Y_real = test_set["gender"].values

In [48]:
Success = Y_pred == Y_real
Accuracy = np.sum(Success)/len(Success)
Accuracy

0.8007316457825246